# Knight move on keypad

## Import functions

In [1]:
import sys, os

# Add the parent directory to the sys.path
if "__file__" in globals().keys():
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
else:
    sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
from src.keypad_data import get_keypad_data
from src.positions import get_vowel_positions, get_missing_positions, get_coordinate_bounds
from src.sequence_check import check_inside_bounds, check_in_avoided_coordinate
from src.sequence_create import get_all_sequences
from src.vowel_check import has_at_most_two_vowels

## Initial tests

In [3]:
# Create the main coordinate data containing the letter/number grid
# This creates coordinate tuples in format (row, col, value) for a 4x5 grid
coordinate_data = get_keypad_data()
print("Keypad (Coordinate Data):")
print("First few coordinates:")
for i, coord in enumerate(coordinate_data[:10]):
    print(f"  {coord}")
print(f"  ... and {len(coordinate_data) - 10} more coordinates")
print()

# Find all missing/empty positions in the coordinate data
# These positions will be avoided during sequence generation
miss_pos = get_missing_positions(coordinate_data)
print(f"Missing positions (to avoid): {miss_pos}")

# Find all vowel positions (A, E, I, O, U) in the coordinate data  
# These are special positions that we want to limit visits to
vo_pos = get_vowel_positions(coordinate_data)
print(f"Vowel positions (limited visits): {vo_pos}")

# Extract the numeric bounds of the coordinate data
# This gives us [x_min, x_max, y_min, y_max] for boundary checking
bounds_dict = get_coordinate_bounds(coordinate_data)
idx_bounds = [bounds_dict['x_min'], bounds_dict['x_max'], bounds_dict['y_min'], bounds_dict['y_max']]
print(f"Index bounds [x_min, x_max, y_min, y_max]: {idx_bounds}")
print()

Keypad (Coordinate Data):
First few coordinates:
  (1, 1, 'A')
  (1, 2, 'B')
  (1, 3, 'C')
  (1, 4, 'D')
  (1, 5, 'E')
  (2, 1, 'F')
  (2, 2, 'G')
  (2, 3, 'H')
  (2, 4, 'I')
  (2, 5, 'J')
  ... and 10 more coordinates

Missing positions (to avoid): [(4, 1), (4, 5)]
Vowel positions (limited visits): [(1, 1), (1, 5), (2, 4), (3, 5)]
Index bounds [x_min, x_max, y_min, y_max]: [1, 4, 1, 5]



In [4]:
# Create a filter function that validates knight move sequences
# A valid sequence must:
# 1. Stay within the coordinate boundaries (check_inside_bounds)
# 2. Avoid all missing/empty positions (not check_in_avoided_coordinate)
filter = lambda sequence: (check_inside_bounds(sequence, *idx_bounds)) and (not check_in_avoided_coordinate(sequence, miss_pos))

In [5]:
# Test sequences
sequences = get_all_sequences(1, 1, 3, location_list=vo_pos, max_hit=2,
               filter=filter,
               )

print(f"Total number of sequences: {len(sequences)}")
print("Sequences:")

# Create a lookup dictionary for coordinate values
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

for sequence in sequences:
    sequence_string = ("".join([coord_lookup[i, j] for i, j in sequence]))
    print(sequence)
    print(sequence_string)

Total number of sequences: 28
Sequences:
[(1, 1), (3, 2), (1, 3), (3, 4)]
ALCN
[(1, 1), (3, 2), (1, 3), (3, 2)]
ALCL
[(1, 1), (3, 2), (1, 3), (2, 5)]
ALCJ
[(1, 1), (3, 2), (1, 3), (2, 1)]
ALCF
[(1, 1), (3, 2), (1, 1), (3, 2)]
ALAL
[(1, 1), (3, 2), (1, 1), (2, 3)]
ALAH
[(1, 1), (3, 2), (4, 4), (2, 5)]
AL3J
[(1, 1), (3, 2), (4, 4), (2, 3)]
AL3H
[(1, 1), (3, 2), (4, 4), (3, 2)]
AL3L
[(1, 1), (3, 2), (2, 4), (4, 3)]
ALI2
[(1, 1), (3, 2), (2, 4), (3, 2)]
ALIL
[(1, 1), (3, 2), (2, 4), (1, 2)]
ALIB
[(1, 1), (2, 3), (4, 4), (2, 5)]
AH3J
[(1, 1), (2, 3), (4, 4), (2, 3)]
AH3H
[(1, 1), (2, 3), (4, 4), (3, 2)]
AH3L
[(1, 1), (2, 3), (4, 2), (2, 3)]
AH1H
[(1, 1), (2, 3), (4, 2), (2, 1)]
AH1F
[(1, 1), (2, 3), (4, 2), (3, 4)]
AH1N
[(1, 1), (2, 3), (3, 5), (1, 4)]
AHOD
[(1, 1), (2, 3), (3, 5), (4, 3)]
AHO2
[(1, 1), (2, 3), (3, 5), (2, 3)]
AHOH
[(1, 1), (2, 3), (3, 1), (1, 2)]
AHKB
[(1, 1), (2, 3), (3, 1), (4, 3)]
AHK2
[(1, 1), (2, 3), (3, 1), (2, 3)]
AHKH
[(1, 1), (2, 3), (1, 5), (3, 4)]
AHEN
[(1, 1), 

In [6]:
# Check the total number of sequences created with and without filtering
#
# Purpose: Compare two approaches:
# 1) Create all sequences and then apply `filter` afterwards
# 2) Generate sequences while applying the `filter` during creation
# If both approaches are equivalent, they should yield the same sequence lists.

# Local function for testing
def sequence_count_check(sequence_list):
    total_count = len(sequence_list)
    inside_count = len([sequence for sequence in sequence_list if check_inside_bounds(sequence, *idx_bounds)])
    outside_count = total_count - inside_count
    in_avoided_count = len([sequence for sequence in sequence_list if check_in_avoided_coordinate(sequence, miss_pos)])
    not_in_avoided_count = total_count - in_avoided_count

    print(f"Total number of sequences: {total_count}")
    print(f"Total number of sequences inside and outside the bounds: {inside_count + outside_count}")
    print(f"Total number of sequences hitting or not hitting the avoided positions: {in_avoided_count + not_in_avoided_count}")

# Starting coordinates and step count (number of knight moves)
x_i = 1
y_i = 1
step_i = 3

print("Create all sequences then apply filter:")

# 1) Generate all possible sequences (no filter applied during generation)
all_sequence_list = get_all_sequences(x_i, y_i, step_i)

# Diagnostics: show counts of sequences inside/outside bounds and hitting/missing positions
sequence_count_check(all_sequence_list)

# Apply the filter function to the pre-generated list
sequence_list_after_filter = [sequence for sequence in all_sequence_list if filter(sequence)]
print(f"Total number of sequences after applying the filter: {len(sequence_list_after_filter)}")

print("Create all sequences with filter:")

# 2) Generate sequences while applying the filter during generation (more memory-efficient)
all_sequence_list = get_all_sequences(x_i, y_i, step_i, filter=filter)

# Diagnostics for the second method
sequence_count_check(all_sequence_list)

# Collect sequences produced when filter is applied during generation
sequence_list_with_filter = [sequence for sequence in all_sequence_list if filter(sequence)]
print(f"Total number of sequences: {len(sequence_list_with_filter)}")

# Compare results from both approaches — they should be equal if filter semantics match
print(f"Do both methods yield the same sequences: {sequence_list_after_filter == sequence_list_with_filter}")

if sequence_list_after_filter == sequence_list_with_filter:

    # Create a lookup dictionary for coordinate values to produce readable sequence strings
    coord_lookup = {(x, y): value for x, y, value in coordinate_data}

    # Uncomment to show each sequence and its mapped string representation
    print("\nSequence strings from sequence_list_with_filter:")
    for i, sequence in enumerate(sequence_list_with_filter):
        # Build a string from coordinate tuples using the lookup table
        sequence_string = "".join([coord_lookup[i, j] for i, j in sequence])
        print(f"Sequence {i+1}: {sequence} -> '{sequence_string}'")

Create all sequences then apply filter:
Total number of sequences: 512
Total number of sequences inside and outside the bounds: 512
Total number of sequences hitting or not hitting the avoided positions: 512
Total number of sequences after applying the filter: 28
Create all sequences with filter:
Total number of sequences: 28
Total number of sequences inside and outside the bounds: 28
Total number of sequences hitting or not hitting the avoided positions: 28
Total number of sequences: 28
Do both methods yield the same sequences: True

Sequence strings from sequence_list_with_filter:
Sequence 1: [(1, 1), (3, 2), (1, 3), (3, 4)] -> 'ALCN'
Sequence 2: [(1, 1), (3, 2), (1, 3), (3, 2)] -> 'ALCL'
Sequence 3: [(1, 1), (3, 2), (1, 3), (2, 5)] -> 'ALCJ'
Sequence 4: [(1, 1), (3, 2), (1, 3), (2, 1)] -> 'ALCF'
Sequence 5: [(1, 1), (3, 2), (1, 1), (3, 2)] -> 'ALAL'
Sequence 6: [(1, 1), (3, 2), (1, 1), (2, 3)] -> 'ALAH'
Sequence 7: [(1, 1), (3, 2), (4, 4), (2, 5)] -> 'AL3J'
Sequence 8: [(1, 1), (3, 

## Efficiency analysis

In [7]:
import time
from datetime import datetime

def time_execution(func, *args, description="Function execution", **kwargs):
    """
    Time the execution of a function and return results with timing information.
    
    Parameters:
    -----------
    func : callable
        The function to time
    *args : tuple
        Positional arguments for the function
    description : str
        Description of what's being timed
    **kwargs : dict
        Keyword arguments for the function
    
    Returns:
    --------
    tuple
        (result, execution_time_seconds)
    """
    print(f"\n🕐 Starting: {description}")
    print(f"📅 Start time: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
    
    start_time = time.perf_counter()
    result = func(*args, **kwargs)
    end_time = time.perf_counter()
    
    execution_time = end_time - start_time
    
    print(f"⏱️  Execution time: {execution_time:.4f} seconds")
    print(f"📅 End time: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
    
    if execution_time < 0.001:
        print(f"⚡ Very fast: {execution_time*1000:.2f} milliseconds")
    elif execution_time < 1:
        print(f"🏃 Fast: {execution_time*1000:.1f} milliseconds") 
    elif execution_time < 10:
        print(f"🚶 Moderate: {execution_time:.2f} seconds")
    else:
        print(f"🐌 Slow: {execution_time:.1f} seconds")
    
    return result, execution_time

# Time the sequence generation for different steps
print("="*60)
print("TIMING ANALYSIS FOR SEQUENCE GENERATION")
print("="*60)

step_i = 5
x_i, y_i = 1, 1
max_hit = 2

# Time different operations
timing_results = {}

# 1. Time basic sequence generation
sequences, time1 = time_execution(
    get_all_sequences, x_i, y_i, step_i,
    description=f"Basic sequence generation ({step_i} moves)"
)
timing_results['basic_sequences'] = (len(sequences), time1)

# 2. Time sequence generation with vowel location constraints
sequences_vowel, time2 = time_execution(
    get_all_sequences, x_i, y_i, step_i,
    location_list=vo_pos, max_hit=max_hit,
    description=f"Sequence generation with vowel constraints ({step_i} moves)"
)
timing_results['vowel_constrained'] = (len(sequences_vowel), time2)

# 3. Time sequence generation with filter
sequences_filtered, time3 = time_execution(
    get_all_sequences, x_i, y_i, step_i,
    location_list=vo_pos, max_hit=max_hit, filter=filter,
    description=f"Sequence generation with filter ({step_i} moves)"
)
timing_results['filtered'] = (len(sequences_filtered), time3)

# 4. Time the vowel checking loop
def vowel_check_loop():
    total_sequences = 0
    # Create a lookup dictionary for coordinate values
    coord_lookup = {(x, y): value for x, y, value in coordinate_data}
    
    for x_i in range(1, 5):
        for y_i in range(1, 6):
            all_sequence_list = get_all_sequences(x_i, y_i, step_i, location_list=vo_pos, max_hit=max_hit, filter=filter)
            for sequence in all_sequence_list:
                sequence_string = "".join([coord_lookup[i, j] for i, j in sequence])
                if has_at_most_two_vowels(sequence_string):
                    total_sequences += 1
    return total_sequences

vowel_count, time4 = time_execution(
    vowel_check_loop,
    description=f"Complete vowel analysis loop ({step_i} moves)"
)
timing_results['vowel_analysis'] = (vowel_count, time4)

# Display summary
print("\n" + "="*60)
print("TIMING SUMMARY")
print("="*60)
print(f"{'Operation':<35} {'Sequences':<10} {'Time (s)':<12} {'Sequences/sec':<12}")
print("-" * 60)

for operation, (seq_count, exec_time) in timing_results.items():
    rate = seq_count / exec_time if exec_time > 0 else 0
    print(f"{operation:<35} {seq_count:<10} {exec_time:<12.4f} {rate:<12.1f}")

print("\n📊 Performance Insights:")
print(f"• Filter reduces sequences by {((timing_results['basic_sequences'][0] - timing_results['filtered'][0]) / timing_results['basic_sequences'][0] * 100):.1f}%")
print(f"• Vowel constraints reduce sequences by {((timing_results['basic_sequences'][0] - timing_results['vowel_constrained'][0]) / timing_results['basic_sequences'][0] * 100):.1f}%")
print(f"• Processing rate: {timing_results['vowel_analysis'][0] / timing_results['vowel_analysis'][1]:.1f} vowel checks per second")


TIMING ANALYSIS FOR SEQUENCE GENERATION

🕐 Starting: Basic sequence generation (5 moves)
📅 Start time: 14:34:44.161
⏱️  Execution time: 0.2461 seconds
📅 End time: 14:34:44.408
🏃 Fast: 246.1 milliseconds

🕐 Starting: Sequence generation with vowel constraints (5 moves)
📅 Start time: 14:34:44.409
⏱️  Execution time: 0.1400 seconds
📅 End time: 14:34:44.549
🏃 Fast: 140.0 milliseconds

🕐 Starting: Sequence generation with filter (5 moves)
📅 Start time: 14:34:44.549
⏱️  Execution time: 0.0058 seconds
📅 End time: 14:34:44.556
🏃 Fast: 5.8 milliseconds

🕐 Starting: Complete vowel analysis loop (5 moves)
📅 Start time: 14:34:44.556
⏱️  Execution time: 0.1565 seconds
📅 End time: 14:34:44.713
🏃 Fast: 156.5 milliseconds

TIMING SUMMARY
Operation                           Sequences  Time (s)     Sequences/sec
------------------------------------------------------------
basic_sequences                     32768      0.2461       133157.7    
vowel_constrained                   31384      0.1400       

## Test different methods to get the sequences with limited number of vowels

In [8]:
import time

x_min, x_max = bounds_dict['x_min'], bounds_dict['x_max']
y_min, y_max = bounds_dict['y_min'], bounds_dict['y_max']
max_hit = 2

### Apply filter and limit the number of vowels when creating the sequences

In [9]:
# Iterate through different sequence lengths and total_sequences with timing

print("Sequence Analysis with Timing:")
print("=" * 60)
print(f"{'Step':<6} {'Sequence Length':<12} {'Total Sequences':<12} {'Time (s)':<10} {'Seq/sec':<10}")
print("-" * 60)

for step_i in range(3, 10):
    start_time = time.perf_counter()
    
    total_sequences = 0
    for x_i in range(x_min, x_max + 1):
        for y_i in range(y_min, y_max + 1):
            all_sequence_list = get_all_sequences(x_i, y_i, step_i, location_list=vo_pos, max_hit=max_hit, filter=filter)
            total_sequences += len(all_sequence_list)
    
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    sequence_length = step_i + 1  # step_i is number of moves, sequence length is moves + 1
    seq_per_sec = total_sequences / execution_time if execution_time > 0 else 0
   
    print(f"{step_i:<6} {sequence_length:<12} {total_sequences:<12} {execution_time:<10.4f} {seq_per_sec:<10.1f}")

print("=" * 60)


Sequence Analysis with Timing:
Step   Sequence Length Total Sequences Time (s)   Seq/sec   
------------------------------------------------------------
3      4            732          0.0158     46198.4   
4      5            2486         0.0407     61030.5   
5      6            8392         0.1593     52691.7   
6      7            27912        0.7584     36804.2   
7      8            93204        2.6030     35806.8   
8      9            306288       10.0606    30444.3   
9      10           1013398      43.8089    23132.2   


### Apply filter and limit the number of vowels after creating the sequences

**Note: Beware of the memory limit. My laptop cannot handle more then 8 moves (sequence of 9)

In [10]:
# Iterate through different sequence lengths with vowel checking and timing

print("Sequence Analysis with Vowel Checking and Timing:")
print("=" * 70)
print(f"{'Step':<6} {'Sequence Length':<12} {'Total Sequences':<12} {'Time (s)':<10} {'Seq/sec':<10}")
print("-" * 70)

# Create a lookup dictionary for coordinate values (once, outside the loop)
coord_lookup = {(x, y): value for x, y, value in coordinate_data}

for step_i in range(3, 9):
    start_time = time.perf_counter()
    
    total_sequences = 0
    for x_i in range(x_min, x_max + 1):
        for y_i in range(y_min, y_max + 1):

            all_sequence_list = get_all_sequences(x_i, y_i, step_i)
            sequence_list_final = [sequence for sequence in all_sequence_list if filter(sequence)]

            for sequence in sequence_list_final:
                sequence_string = ("".join([coord_lookup[i, j] for i, j in sequence]))
                
                if has_at_most_two_vowels(sequence_string):
                    total_sequences += 1

    end_time = time.perf_counter()
    execution_time = end_time - start_time
    sequence_length = step_i + 1  # step_i is number of moves, sequence length is moves + 1
    seq_per_sec = total_sequences / execution_time if execution_time > 0 else 0
    
    print(f"{step_i:<6} {sequence_length:<12} {total_sequences:<12} {execution_time:<10.4f} {seq_per_sec:<10.1f}")

print("=" * 70)


Sequence Analysis with Vowel Checking and Timing:
Step   Sequence Length Total Sequences Time (s)   Seq/sec   
----------------------------------------------------------------------
3      4            732          0.0794     9216.9    
4      5            2486         0.4512     5510.1    
5      6            8392         3.6928     2272.5    
6      7            27912        32.9045    848.3     
7      8            93204        272.2165   342.4     
8      9            306288       2153.6724  142.2     
